In [ ]:
import pandas as pd
import openpyxl
import re
import os
from datetime import datetime, timedelta

# --- CONFIGURACIÓN ---
# Palabras clave para detectar las secciones (igual que antes)
PALABRAS_CLAVE = {
    'TRA_END': ['TOTALES TRA', 'TOTALES TRADICIONAL'],
    'HP_START': ['TOTALES HP', 'TOTALES HOJA'],
    'BORR_START': ['TOTAL BORR', 'TOTALES BORR', 'TOTALES BORRACHO'],
    'BORR_END': ['TOTAL ELOTE', 'ELOTE']
}

def calcular_fecha_inicio(numero_semana):
    """
    Calcula el lunes de inicio basado en la regla:
    Semana 1 = 29 de Diciembre de 2025.
    """
    try:
        num = int(numero_semana)
        fecha_base = datetime(2025, 12, 29) # Lunes de Sem #1
        # Sumar 7 días por cada semana extra (Sem 2 es base + 7 días, etc.)
        fecha_inicio = fecha_base + timedelta(days=(num - 1) * 7)
        return fecha_inicio
    except ValueError:
        return None

def limpiar_texto(texto):
    if pd.isna(texto): return ""
    return re.sub(r'\s+', ' ', str(texto)).strip().upper()

def buscar_fila_clave(df, lista_palabras, n_cols=5):
    """Busca filas delimitadoras en las primeras columnas."""
    for col_idx in range(n_cols):
        if col_idx >= len(df.columns): break
        col_str = df.iloc[:, col_idx].apply(limpiar_texto)
        for palabra in lista_palabras:
            matches = col_str[col_str.str.contains(palabra, na=False)]
            if not matches.empty:
                return matches.index[0]
    return None

def identificar_guiso(texto):
    """Estandariza nombres de guisos."""
    t = limpiar_texto(texto).replace('.', '')
    mapeo = {
        'F': 'FRIJOL', 'FRIJOL': 'FRIJOL', 'FRIJO': 'FRIJOL',
        'P': 'POLLO', 'POLLO': 'POLLO', 'POLL': 'POLLO',
        'Q': 'QUESO', 'QUESO': 'QUESO', 'QUES': 'QUESO',
        'PCO': 'PCO', 'PUERCO': 'PCO', 'CERDO': 'PCO',
        'SV': 'SALSA VERDE', 'VERDE': 'SALSA VERDE', 'SALSA': 'SALSA VERDE',
        'DULCE': 'DULCE', 'RAJAS': 'RAJAS', 'CHICHARRON': 'CHICHARRON',
        'PICADILLO': 'PICADILLO'
    }
    return mapeo.get(t, t)

def interpretar_nota(texto_nota, total_celda):
    """Extrae cantidades de notas tipo '8 frijol y 4 pco'."""
    if not texto_nota: return None
    texto = texto_nota.lower().replace('\n', ' ')
    resultados = {}

    patrones = [r'(\d+)\s*(?:de)?\s*([a-z\.]+)', r'([a-z\.]+)\s*:?\s*(\d+)']

    for p in patrones:
        for match in re.findall(p, texto):
            try:
                cant = float(match[0])
                palabra = match[1]
            except:
                try:
                    cant = float(match[1])
                    palabra = match[0]
                except: continue

            guiso = identificar_guiso(palabra)
            if guiso and guiso not in ['DE', 'Y', 'EL', 'LA', 'CAJAS', 'TOTAL']:
                resultados[guiso] = resultados.get(guiso, 0) + cant

    if resultados and abs(sum(resultados.values()) - total_celda) <= 2:
        return resultados
    return None

def procesar_nuevo_ciclo():
    print("--- PROCESANDO NUEVO CICLO (SEM #1 -> 29 DIC) ---")

    archivos = [f for f in os.listdir('.') if f.endswith('.xlsx') and not f.startswith('~$')]
    if not archivos:
        print("ERROR: No se encuentra archivo .xlsx.")
        return

    # Procesamos todos los excels encontrados
    datos_totales = []

    for archivo in archivos:
        print(f"Leyendo archivo: {archivo}")
        try:
            wb = openpyxl.load_workbook(archivo, data_only=True)
        except Exception as e:
            print(f"Error abriendo {archivo}: {e}")
            continue

        for hoja in wb.sheetnames:
            # Buscar número de semana (ej. "Sem 1", "Sem #01", "1")
            match = re.search(r'SEM.*?(\d+)', hoja.upper())
            if not match:
                # Intentar buscar solo el número si la hoja se llama "1", "2"
                if hoja.strip().isdigit():
                    num_sem = int(hoja)
                else:
                    continue
            else:
                num_sem = int(match.group(1))

            # Calcular fecha
            fecha_inicio = calcular_fecha_inicio(num_sem)
            print(f"  -> Hoja '{hoja}' detectada como Semana {num_sem} (Inicio: {fecha_inicio.date()})")

            ws = wb[hoja]
            data = list(ws.values)
            if not data: continue
            df = pd.DataFrame(data)

            # Buscar límites de tablas
            idx_tra_end = buscar_fila_clave(df, PALABRAS_CLAVE['TRA_END'])
            idx_hp_start = buscar_fila_clave(df, PALABRAS_CLAVE['HP_START'])
            idx_borr_start = buscar_fila_clave(df, PALABRAS_CLAVE['BORR_START'])
            idx_borr_end = buscar_fila_clave(df, PALABRAS_CLAVE['BORR_END']) or len(df)

            if None in [idx_tra_end, idx_hp_start, idx_borr_start]:
                print(f"     AVISO: No se detectó la estructura completa (TRA/HP/BORR) en '{hoja}'. Saltando.")
                continue

            bloques = [
                (range(3, idx_tra_end), "Tradicional"),
                (range(idx_hp_start+1, idx_borr_start), "Hoja de Platano (HP)"),
                (range(idx_borr_start+1, idx_borr_end), "Borracho")
            ]

            count_reg = 0
            for rango, tipo in bloques:
                for r_idx in rango:
                    fila = df.iloc[r_idx]
                    nombre = fila[0]
                    if pd.isna(nombre) or "TOTAL" in str(nombre).upper(): continue

                    for i in range(7): # 7 días
                        c_guiso = 2 + (i * 3)
                        c_cant = 3 + (i * 3)
                        if c_cant >= len(fila): continue

                        try: cant = float(fila[c_cant])
                        except: cant = 0

                        if cant > 0:
                            guiso_raw = limpiar_texto(fila[c_guiso])
                            guiso_final = guiso_raw if guiso_raw else "DESCONOCIDO"
                            registrado = False

                            # Manejo de mezclas con notas
                            if '/' in guiso_raw:
                                celda = ws.cell(row=r_idx+1, column=c_guiso+1)
                                if celda.comment:
                                    desglose = interpretar_nota(celda.comment.text, cant)
                                    if desglose:
                                        registrado = True
                                        for g, c in desglose.items():
                                            datos_totales.append({
                                                'Fecha': fecha_inicio + timedelta(days=i),
                                                'Tipo_Tamal': tipo, 'Guiso': g, 'Cantidad': c
                                            })

                                # Fallback división simple
                                if not registrado:
                                    partes = [p for p in guiso_raw.split('/') if p.strip()]
                                    if partes:
                                        split = round(cant / len(partes), 2)
                                        for p in partes:
                                            datos_totales.append({
                                                'Fecha': fecha_inicio + timedelta(days=i),
                                                'Tipo_Tamal': tipo, 'Guiso': identificar_guiso(p), 'Cantidad': split
                                            })
                                        registrado = True

                            # Registro normal
                            if not registrado:
                                datos_totales.append({
                                    'Fecha': fecha_inicio + timedelta(days=i),
                                    'Tipo_Tamal': tipo,
                                    'Guiso': identificar_guiso(guiso_final),
                                    'Cantidad': cant
                                })
                            count_reg += 1
            print(f"     -> Registros extraídos: {count_reg}")

    # Guardar CSV final
    if datos_totales:
        df_fin = pd.DataFrame(datos_totales)
        df_agrupado = df_fin.groupby(['Fecha', 'Tipo_Tamal', 'Guiso'], as_index=False)['Cantidad'].sum()
        df_agrupado.sort_values(by=['Fecha', 'Tipo_Tamal', 'Guiso'], inplace=True)

        archivo_salida = "produccion_nuevo_ciclo.csv"
        df_agrupado.to_csv(archivo_salida, index=False)
        print("\n¡ÉXITO! Archivo generado:", archivo_salida)
        print(df_agrupado.head(10))
    else:
        print("No se generaron datos.")

procesar_nuevo_ciclo()

--- PROCESANDO NUEVO CICLO (SEM #1 -> 29 DIC) ---
Leyendo archivo: SEMANAS  (1).xlsx
  -> Hoja ' Sem #01' detectada como Semana 1 (Inicio: 2025-12-29)
     -> Registros extraídos: 86
  -> Hoja ' Sem #02' detectada como Semana 2 (Inicio: 2026-01-05)
     -> Registros extraídos: 97
  -> Hoja ' Sem #03' detectada como Semana 3 (Inicio: 2026-01-12)
     -> Registros extraídos: 16

¡ÉXITO! Archivo generado: produccion_nuevo_ciclo.csv
       Fecha            Tipo_Tamal        Guiso  Cantidad
0 2025-12-29              Borracho          PCO      17.0
1 2025-12-29              Borracho        QUESO       3.0
2 2025-12-29              Borracho  SALSA VERDE       7.0
3 2025-12-29  Hoja de Platano (HP)          PCO     906.0
4 2025-12-29  Hoja de Platano (HP)        POLLO     326.0
5 2025-12-29           Tradicional       FRIJOL      15.0
6 2025-12-29           Tradicional          PCO      58.0
7 2025-12-29           Tradicional        POLLO      24.0
8 2025-12-29           Tradicional        QUE